In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import copy
import cv2

In [26]:
def conditions(s):
    if (s['Wind']>=25) and (s['Wind']<30):
        return 1.0
    elif (s['Wind']>=30) and (s['Wind']<35):
        return 2.0
    elif (s['Wind']>=35) and (s['Wind']<45):
        return 2.5
    elif (s['Wind']>=45) and (s['Wind']<55):
        return 3.0
    elif (s['Wind']>=55) and (s['Wind']<65):
        return 3.5
    elif (s['Wind']>=65) and (s['Wind']<77):
        return 4.0
    elif (s['Wind']>=77) and (s['Wind']<90):
        return 4.5
    elif (s['Wind']>=90) and (s['Wind']<102):
        return 5.0
    elif (s['Wind']>=102) and (s['Wind']<115):
        return 5.5
    elif (s['Wind']>=115) and (s['Wind']<127):
        return 6.0            
    elif (s['Wind']>=127) and (s['Wind']<140):
        return 6.5
    elif (s['Wind']>=140) and (s['Wind']<155):
        return 7.0
    elif (s['Wind']>=155) and (s['Wind']<170):
        return 7.5
    elif (s['Wind']>=170) and (s['Wind']<185):
        return 8.0
    else:
        return 8.5
                                  
imgFreq = {}
fList = ['19H','19V','22V','37V','37H','91H','91V','150H','183_1H','183_3H','183_7H']
rList = ['ATL','CPAC','EPAC','IO','SHEM','WPAC']
for r in rList:
    imgFreq[r] = {}
    for f in fList:
        imgFreq[r][f] = pd.read_csv("..//ImagesPerFreq//"+r+"_"+f+'.csv')
        #df["T_No"] = 0
        #df['T_No'] = df.apply(conditions, axis=1)
        
        #df.to_csv("..//ImagesPerFreq//"+r+"_"+f+'.csv',index=False)

In [30]:
imgFreq['ATL']['19V']

,FileName,CenLon,CenLat,Pressure,Wind,Area,T_No
0,..//..//MyCreatedData//5//ATL//29//F16//S1//19...,-40.820101,30.037247,984,45,0.726836,3.0
1,..//..//MyCreatedData//5//ATL//29//F16//S1//19...,-41.221494,27.498088,983,50,0.980471,3.0
2,..//..//MyCreatedData//5//ATL//29//F16//S1//19...,-40.319371,25.653805,982,55,0.986157,3.5
3,..//..//MyCreatedData//5//ATL//29//F16//S1//19...,-39.045968,24.814455,980,60,0.984815,3.5
4,..//..//MyCreatedData//5//ATL//29//F16//S1//19...,-39.561312,23.379201,982,55,0.986157,3.5
...,...,...,...,...,...,...,...
5371,..//..//MyCreatedData//19//ATL//20//F18//S1//1...,-59.266902,23.975245,999,50,0.374884,3.0
5372,..//..//MyCreatedData//19//ATL//20//F18//S1//1...,-53.953452,26.017272,999,50,0.060826,3.0
5373,..//..//MyCreatedData//19//ATL//20//F18//S1//1...,-53.469813,26.245254,998,55,0.035787,3.5
5374,..//..//MyCreatedData//19//ATL//20//F18//S1//1...,-45.966468,31.907491,997,55,0.850694,3.5


In [3]:
class Clustering():
    def __init__(self,x,clusterSize=2):
        self.k = clusterSize
        self.m = x.shape[0]
        self.p = x.shape[1]
        self.x = x
        
    def Fuzzy_K_Means(self):
        self.x = self.x / np.amax(self.x, axis=0)
        
        prevC = np.random.rand(self.k,self.p)
        c = np.zeros((self.k,self.p))
        center = {}
        rand_center_index = random.sample(range(0, self.m), self.k)
        
        # Randomly assigning each cluster center
        # a unique x (whose index lies from 0 to m)
        for j in range(self.k):
            c[j,:] = self.x[rand_center_index[j], :] 
        
        print(c.shape)
        isConverged = False
        apple=0
        while not isConverged and apple<=1000:
            for j in range(self.k):
                center[j] = []
                
            for i in range(self.m):
                # The length of this list will be K same as number of cluster
                similarity = []
                for j in range(self.k):
                    #Calculate Fuzzy Similarity
                    fs = FuzzySimilarity(self.x[i,:],c[j,:])
                    similarity.append( fs )

                # Assigning image(index) to unique cluster based on
                # Max similarity value
                index = similarity.index(max(similarity))
                center[index].append(i)

            # Updating cluster center
            for j in range(self.k):
                total = np.zeros((1,self.p))  
                # Iterating through each images in this unique cluster
                for i in center[j]:
                    total[0,:] += self.x[i,:]
                c[j,:] = total/len(center[j])
                
            if (prevC == c).all():
                isConverged = True
            apple+=1     
            prevC = copy.deepcopy(c)
        
            print(apple)
        return center, c

In [4]:
imgFreq = {}
fList = ['19H','19V','22V','37V','37H','91H','91V','150H','183_1H','183_3H','183_7H']
rList = ['ATL','CPAC','EPAC','IO','SHEM','WPAC']
for r in rList:
    imgFreq[r] = {}
    for f in fList:
        imgFreq[r][f] = pd.read_csv("..//ImagesPerFreq//"+r+"_"+f+'.csv')
        del imgFreq[r][f]['Unnamed: 0']

In [5]:
atl_19h = imgFreq['ATL']['19H'].loc[imgFreq['ATL']['19H']['Area'] >= 0.90]
atl_183_1h = imgFreq['ATL']['183_1H'].loc[imgFreq['ATL']['183_1H']['Area'] >= 0.90]

In [6]:
with open('atl_183_1h_matrix.pkl', 'rb') as file:
    matOfImages = dill.load(file)

In [ ]:
cL = Clustering(matOfImages[:,0:np.size(matOfImages,axis=1)-2],7)
center, c = cL.Fuzzy_K_Means()

Testing

In [ ]:
img_cv = cv2.imread("test.png",cv2.IMREAD_UNCHANGED)  
img_cv = cv2.resize(img_cv, (360,360))
gray_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

In [ ]:
# Output img with window name as 'image' 
cv2.imshow('image', g2)  
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
flatten = gray_cv.flatten()

In [ ]:
newGray = np.reshape(flatten,(360,360))

In [ ]:
g1 = matOfImages[0,0:129600]
g1 = g1/255
g1 = g1*255

In [ ]:
g2 = np.reshape(g1, (360,360)).astype(np.uint8)

In [ ]:
g2

In [ ]:
import shutil
for k,v in center.items():
    for i in v:
        shutil.copy(atl_183_1H.iloc[i,0], '..\\..\\AllFrequencies\\ATL\\183_1H\\'+str(k) )

In [ ]:
newC = np.reshape(matOfImages[0,0:129600], (360,360))
newC.astype(int)